## parser+splitter/process_pdf.py의 결과로 나온 updated_documents 불러오기

In [15]:
import pickle


In [16]:
# updated_dociments.pkl은 최근에 Upstage에서 Document Parser를 업데이트 한 것을 100개의 PDF 파일에 적용시킨 pkl 버전입니다.
with open('updated_documents.pkl', 'rb') as f:
    data = pickle.load(f)

In [39]:
from pprint import pprint
pprint(data[:15])  # 첫 번째 요소 출력


[Document(metadata={'category': 'paragraph', 'coordinates': [{'x': 0.0535, 'y': 0.2032}, {'x': 0.1824, 'y': 0.2032}, {'x': 0.1824, 'y': 0.2368}, {'x': 0.0535, 'y': 0.2368}], 'page': 1, 'id': 0, 'company_name': '크래프톤', 'report_date': '20240509', 'securities_firm': '미래에셋증권', 'source_file': '크래프톤_20240509_미래에셋증권.pdf', 'content_types': {'markdown': 'Equity Research\n2024.5.9', 'text': 'Equity Research\n2024.5.9', 'html': "<p id='0' data-category='paragraph' style='font-size:16px'>Equity Research<br>2024.5.9</p>"}}, page_content='Equity Research\n2024.5.9'),
 Document(metadata={'category': 'table', 'coordinates': [{'x': 0.041, 'y': 0.2669}, {'x': 0.2923, 'y': 0.2669}, {'x': 0.2923, 'y': 0.7877}, {'x': 0.041, 'y': 0.7877}], 'page': 1, 'id': 1, 'company_name': '크래프톤', 'report_date': '20240509', 'securities_firm': '미래에셋증권', 'source_file': '크래프톤_20240509_미래에셋증권.pdf', 'base64_encoding': '...', 'content_types': {'markdown': '| 투자의견(유지) | 매수 |\n| --- | --- |\n| 목표주가(상향) ▲ | 370,000원 |\n| 현재주가(24/5

In [40]:
len(data)

7984

In [42]:
data_page_contents = [item.page_content for item in data]

## 전처리 하기 전 청킹 단위

In [44]:
data_page_contents[:15]

['Equity Research\n2024.5.9',
 '| 투자의견(유지) | 매수 |\n| --- | --- |\n| 목표주가(상향) ▲ | 370,000원 |\n| 현재주가(24/5/8) | 260,000원 |\n| 상승여력 | 42.3% |\n| 영 업이익(24F,십억원) | 801 |\n| Consensus 영업이익(24F,십억원) | 777 |\n| EPS 성장률(24F,%) | 25.1 |\n| MKT EPS 성장률(24F,%) | 75.9 |\n| P/E(24F,x) | 17.0 |\n| MKT P/E(24F,x) | 11.1 |\n| KOSPI | 2,745.05 |\n| 시가총액(십억원) | 12,575 |\n| 발행주식수(백만주) | 48 |\n| 유동주식비율(%) | 58.8 |\n| 외국인 보유비중(%) | 36.5 |\n| 베타(12M) 일간수익률 | 0.66 |\n| 52주 최저가(원) | 146,500 |\n| 52주 최고가(원) | 260,000 |\n| (%) 1M 절대주가 6.8 | 6M 12M 36.3 28.1 |\n| 상대주가 |  |\n| 5.7 | 20.2 17.3 |\n| 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 | 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 |\n',
 '[인터넷/게임]\n임희석\nheeseok.lim@miraeasset.com',
 '259960 · 게임',
 '# 크래프톤',
 '# 24년에도 질주는 계속된다',
 '1Q24 Review: 컨센서스 상회',
 '1분기 매출액 6,660억원(+24% YoY), 영업이익 3,110억원(+10% YoY)을 기록하며\n컨센서스(매출액 5,680억원, 영업이익 2,410억원)를 각 17%, 29% 상회했다. PC\n매출액이 2,440억원(+37% YoY)으로 예상치를 28% 상회했다. 12월 업데이트 이후\n급격히

---
## 전처리 시작

## 전처리 1

In [21]:
from typing import List
from langchain.schema import Document
import pickle

def merge_document_structures(documents: List[Document]) -> List[Document]:
    """
    문서 구조를 분석하여 관련 내용을 합치는 함수
    
    Args:
        documents: 원본 Document 리스트
    Returns:
        구조가 병합된 Document 리스트
    """
    merged_documents = []
    i = 0
    visual_categories = ['table', 'figure', 'chart', 'paragraph']
    
    while i < len(documents):
        doc = documents[i]
        current_category = doc.metadata.get('category', '')
        
        # 다음 문서 확인 (존재하는 경우)
        next_doc = documents[i + 1] if i + 1 < len(documents) else None
        next_category = next_doc.metadata.get('category', '') if next_doc else ''
        
        # Case 1: heading1 다음에 visual element가 오는 경우
        if (current_category == 'heading1' and 
            next_category in visual_categories):
            heading_content = f"Section Title: {doc.page_content}\n\n"
            merged_doc = Document(
                page_content=heading_content + next_doc.page_content,
                metadata=next_doc.metadata
            )
            merged_documents.append(merged_doc)
            i += 2
            
        # Case 2: header 다음에 caption이 오는 경우
        elif (current_category == 'header' and 
              next_category == 'caption'):
            header_content = f"Header: {doc.page_content}\n\n"
            merged_doc = Document(
                page_content=header_content + next_doc.page_content,
                metadata=next_doc.metadata
            )
            merged_documents.append(merged_doc)
            i += 2
            
        # Case 3: caption 다음에 visual element가 오는 경우
        elif (current_category == 'caption' and 
              next_category in ['table', 'figure', 'chart']):
            caption_content = f"Caption: {doc.page_content}\n\n"
            merged_doc = Document(
                page_content=caption_content + next_doc.page_content,
                metadata=next_doc.metadata
            )
            merged_documents.append(merged_doc)
            i += 2
            
        # Case 4: 짧은 paragraph 다음에 visual element가 오는 경우
        elif (current_category == 'paragraph' and 
              len(doc.page_content.strip()) <= 30 and 
              next_category in ['table', 'figure', 'chart']):
            short_para_content = f"Brief Description: {doc.page_content}\n\n"
            merged_doc = Document(
                page_content=short_para_content + next_doc.page_content,
                metadata=next_doc.metadata
            )
            merged_documents.append(merged_doc)
            i += 2
            
        # 그 외의 경우: 기존 문서 유지
        else:
            if (current_category not in ['heading1', 'header'] and 
                len(doc.page_content.strip()) > 1 and 
                'Analyst' not in doc.page_content and 
                '@' not in doc.page_content):
                merged_documents.append(doc)
            i += 1
    
    return merged_documents

# 테스트 코드
def test_merged_documents(documents):
    merged_docs = merge_document_structures(documents)
    print(f"원본 문서 수: {len(documents)}")
    print(f"병합된 문서 수: {len(merged_docs)}")
    
    if merged_docs:
        print("\n병합된 문서 샘플:")
        for doc in merged_docs[:3]:
            print(f"\n카테고리: {doc.metadata.get('category')}")
            print(f"Content 시작 부분:\n{doc.page_content[:300]}...")
            print("-" * 50)

# 테스트 실행
merged_docs = merge_document_structures(data)

## 전처리 1 결과

In [45]:
merged_docs[:15]

[Document(metadata={'category': 'table', 'coordinates': [{'x': 0.041, 'y': 0.2669}, {'x': 0.2923, 'y': 0.2669}, {'x': 0.2923, 'y': 0.7877}, {'x': 0.041, 'y': 0.7877}], 'page': 1, 'id': 1, 'company_name': '크래프톤', 'report_date': '20240509', 'securities_firm': '미래에셋증권', 'source_file': '크래프톤_20240509_미래에셋증권.pdf', 'base64_encoding': '...', 'content_types': {'markdown': '| 투자의견(유지) | 매수 |\n| --- | --- |\n| 목표주가(상향) ▲ | 370,000원 |\n| 현재주가(24/5/8) | 260,000원 |\n| 상승여력 | 42.3% |\n| 영 업이익(24F,십억원) | 801 |\n| Consensus 영업이익(24F,십억원) | 777 |\n| EPS 성장률(24F,%) | 25.1 |\n| MKT EPS 성장률(24F,%) | 75.9 |\n| P/E(24F,x) | 17.0 |\n| MKT P/E(24F,x) | 11.1 |\n| KOSPI | 2,745.05 |\n| 시가총액(십억원) | 12,575 |\n| 발행주식수(백만주) | 48 |\n| 유동주식비율(%) | 58.8 |\n| 외국인 보유비중(%) | 36.5 |\n| 베타(12M) 일간수익률 | 0.66 |\n| 52주 최저가(원) | 146,500 |\n| 52주 최고가(원) | 260,000 |\n| (%) 1M 절대주가 6.8 | 6M 12M 36.3 28.1 |\n| 상대주가 |  |\n| 5.7 | 20.2 17.3 |\n| 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 | 140 크래프톤 KOSPI 130 120

## 전처리 2 결과

In [46]:
merged_docs2 = merge_document_structures(merged_docs)
merged_docs2[:15]

[Document(metadata={'category': 'table', 'coordinates': [{'x': 0.041, 'y': 0.2669}, {'x': 0.2923, 'y': 0.2669}, {'x': 0.2923, 'y': 0.7877}, {'x': 0.041, 'y': 0.7877}], 'page': 1, 'id': 1, 'company_name': '크래프톤', 'report_date': '20240509', 'securities_firm': '미래에셋증권', 'source_file': '크래프톤_20240509_미래에셋증권.pdf', 'base64_encoding': '...', 'content_types': {'markdown': '| 투자의견(유지) | 매수 |\n| --- | --- |\n| 목표주가(상향) ▲ | 370,000원 |\n| 현재주가(24/5/8) | 260,000원 |\n| 상승여력 | 42.3% |\n| 영 업이익(24F,십억원) | 801 |\n| Consensus 영업이익(24F,십억원) | 777 |\n| EPS 성장률(24F,%) | 25.1 |\n| MKT EPS 성장률(24F,%) | 75.9 |\n| P/E(24F,x) | 17.0 |\n| MKT P/E(24F,x) | 11.1 |\n| KOSPI | 2,745.05 |\n| 시가총액(십억원) | 12,575 |\n| 발행주식수(백만주) | 48 |\n| 유동주식비율(%) | 58.8 |\n| 외국인 보유비중(%) | 36.5 |\n| 베타(12M) 일간수익률 | 0.66 |\n| 52주 최저가(원) | 146,500 |\n| 52주 최고가(원) | 260,000 |\n| (%) 1M 절대주가 6.8 | 6M 12M 36.3 28.1 |\n| 상대주가 |  |\n| 5.7 | 20.2 17.3 |\n| 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 | 140 크래프톤 KOSPI 130 120

In [24]:
len(merged_docs2)

4290

## 전처리 3: 길이가 짧고 정보가 없는 paragraph 데이터 삭제
예시) '자료: 미래에셋증권 리서치센터'

In [27]:
# 길이가 30 이하인 paragraph 제거 및 전후 길이 비교
def filter_short_paragraphs(documents: List[Document]) -> List[Document]:
   """
   카테고리가 paragraph이면서 길이가 30 이하인 Document 제거
   
   Args:
       documents: Document 리스트
   Returns:
       필터링된 Document 리스트
   """
   before_length = len(documents)
   
   filtered_docs = [
       doc for doc in documents 
       if not (doc.metadata.get('category') == 'paragraph' and 
               len(doc.page_content.strip()) <= 30)
   ]
   
   after_length = len(filtered_docs)
   
   print(f"필터링 전 문서 수: {before_length}")
   print(f"필터링 후 문서 수: {after_length}")
   print(f"제거된 문서 수: {before_length - after_length}")
   
   return filtered_docs

# 실행
filtered_docs = filter_short_paragraphs(merged_docs2)

필터링 전 문서 수: 4290
필터링 후 문서 수: 3601
제거된 문서 수: 689


## 전처리 3 결과

In [47]:
filtered_docs[:15]

[Document(metadata={'category': 'table', 'coordinates': [{'x': 0.041, 'y': 0.2669}, {'x': 0.2923, 'y': 0.2669}, {'x': 0.2923, 'y': 0.7877}, {'x': 0.041, 'y': 0.7877}], 'page': 1, 'id': 1, 'company_name': '크래프톤', 'report_date': '20240509', 'securities_firm': '미래에셋증권', 'source_file': '크래프톤_20240509_미래에셋증권.pdf', 'base64_encoding': '...', 'content_types': {'markdown': '| 투자의견(유지) | 매수 |\n| --- | --- |\n| 목표주가(상향) ▲ | 370,000원 |\n| 현재주가(24/5/8) | 260,000원 |\n| 상승여력 | 42.3% |\n| 영 업이익(24F,십억원) | 801 |\n| Consensus 영업이익(24F,십억원) | 777 |\n| EPS 성장률(24F,%) | 25.1 |\n| MKT EPS 성장률(24F,%) | 75.9 |\n| P/E(24F,x) | 17.0 |\n| MKT P/E(24F,x) | 11.1 |\n| KOSPI | 2,745.05 |\n| 시가총액(십억원) | 12,575 |\n| 발행주식수(백만주) | 48 |\n| 유동주식비율(%) | 58.8 |\n| 외국인 보유비중(%) | 36.5 |\n| 베타(12M) 일간수익률 | 0.66 |\n| 52주 최저가(원) | 146,500 |\n| 52주 최고가(원) | 260,000 |\n| (%) 1M 절대주가 6.8 | 6M 12M 36.3 28.1 |\n| 상대주가 |  |\n| 5.7 | 20.2 17.3 |\n| 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 | 140 크래프톤 KOSPI 130 120

## 전처리 4: 구조화된 데이터는 유지하고 본문 내용에 대해서만 split
### 전처리 4 결과 포함

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents_selectively(documents: List[Document], chunk_size: int = 600, chunk_overlap: int = 50) -> List[Document]:
   """
   table, chart, figure는 유지하고 나머지 문서들만 분할하는 함수
   
   Args:
       documents: Document 리스트
       chunk_size: 청크 크기 (기본값: 600)
       chunk_overlap: 청크 중복 크기 (기본값: 50)
   Returns:
       선택적으로 분할된 Document 리스트
   """
   before_length = len(documents)
   
   # 텍스트 분할기 설정
   text_splitter = RecursiveCharacterTextSplitter(
       chunk_size=chunk_size,
       chunk_overlap=chunk_overlap,
       length_function=len
   )
   
   result_documents = []
   
   # 카테고리별 문서 수 계산을 위한 카운터
   before_counts = {}
   after_counts = {}
   
   # 분할 전 카테고리별 카운트
   for doc in documents:
       cat = doc.metadata.get('category', 'unknown')
       before_counts[cat] = before_counts.get(cat, 0) + 1
   
   for doc in documents:
       category = doc.metadata.get('category', '')
       
       # table, chart, figure는 분할하지 않고 그대로 유지
       if category in ['table', 'chart', 'figure']:
           result_documents.append(doc)
       else:
           # 나머지 카테고리는 분할
           splits = text_splitter.split_documents([doc])
           result_documents.extend(splits)
   
   # 분할 후 카테고리별 카운트
   for doc in result_documents:
       cat = doc.metadata.get('category', 'unknown')
       after_counts[cat] = after_counts.get(cat, 0) + 1
   
   after_length = len(result_documents)
   
   # 결과 출력
   print(f"\n전체 문서 수 변화:")
   print(f"분할 전: {before_length}")
   print(f"분할 후: {after_length}")
   print(f"증가된 문서 수: {after_length - before_length}")
   
   print("\n카테고리별 문서 수 변화:")
   all_categories = set(list(before_counts.keys()) + list(after_counts.keys()))
   for cat in sorted(all_categories):
       before = before_counts.get(cat, 0)
       after = after_counts.get(cat, 0)
       print(f"{cat}: {before} -> {after} ({after - before:+d})")
   
   return result_documents

# 실행
split_docs = split_documents_selectively(filtered_docs)


전체 문서 수 변화:
분할 전: 3601
분할 후: 3629
증가된 문서 수: 28

카테고리별 문서 수 변화:
caption: 7 -> 7 (+0)
chart: 598 -> 598 (+0)
figure: 167 -> 167 (+0)
footer: 306 -> 306 (+0)
list: 236 -> 251 (+15)
paragraph: 1201 -> 1214 (+13)
table: 1086 -> 1086 (+0)


## 전처리 5: 메타데이터(회사이름, 발행날짜, 증권사 이름)를 활용
### 전처리 5 결과 포함

In [32]:
def add_metadata_to_content(documents: List[Document]) -> List[Document]:
   """
   Document의 메타데이터 정보를 page_content 앞부분에 추가
   
   Args:
       documents: Document 리스트
   Returns:
       메타데이터가 content에 추가된 Document 리스트
   """
   result_documents = []
   
   for doc in documents:
       # 메타데이터에서 필요한 정보 추출
       company_name = doc.metadata.get('company_name', '')
       report_date = doc.metadata.get('report_date', '')
       securities_firm = doc.metadata.get('securities_firm', '')
       
       # 모든 메타데이터가 존재하는 경우에만 컨텍스트 문장 생성
       if all([company_name, report_date, securities_firm]):
           context = f"{report_date}에 {securities_firm}에서 발행한 {company_name}에 관한 레포트에서 나온 내용.\n\n"
           
           # 새로운 Document 생성 (기존 메타데이터 유지)
           enhanced_doc = Document(
               page_content=context + doc.page_content,
               metadata=doc.metadata
           )
           result_documents.append(enhanced_doc)
       else:
           # 필요한 메타데이터가 없는 경우 원본 유지
           result_documents.append(doc)
   
   # 변경 전후 샘플 출력
   print("처리된 문서 수:", len(result_documents))
   if result_documents:
       print("\n첫 번째 문서 샘플:")
       print("카테고리:", result_documents[0].metadata.get('category'))
       print("Content 시작 부분:\n", result_documents[0].page_content[:300])
       
       # 메타데이터 포함 여부 확인
       has_metadata = sum(1 for doc in result_documents 
                        if all(key in doc.metadata 
                              for key in ['company_name', 'report_date', 'securities_firm']))
       print(f"\n필요한 메타데이터를 모두 포함한 문서 수: {has_metadata}")
       print(f"메타데이터가 불완전한 문서 수: {len(result_documents) - has_metadata}")
   
   return result_documents

# 실행
enhanced_docs = add_metadata_to_content(split_docs)

처리된 문서 수: 3629

첫 번째 문서 샘플:
카테고리: table
Content 시작 부분:
 20240509에 미래에셋증권에서 발행한 크래프톤에 관한 레포트에서 나온 내용.

Brief Description: Equity Research
2024.5.9

| 투자의견(유지) | 매수 |
| --- | --- |
| 목표주가(상향) ▲ | 370,000원 |
| 현재주가(24/5/8) | 260,000원 |
| 상승여력 | 42.3% |
| 영 업이익(24F,십억원) | 801 |
| Consensus 영업이익(24F,십억원) | 777 |
| EPS 성장률(24F,%) | 25.1 |
| MKT EPS 성장률(24F,%) 

필요한 메타데이터를 모두 포함한 문서 수: 3629
메타데이터가 불완전한 문서 수: 0


## 전처리 6: 메타데이터에 uuid와 nid 부여
### 전처리 6 결과 포함

In [35]:
import uuid

def add_unique_identifiers(documents: List[Document]) -> List[Document]:
   """
   각 Document에 uuid와 nid(순차적 번호) 부여
   
   Args:
       documents: Document 리스트
   Returns:
       uuid와 nid가 메타데이터에 추가된 Document 리스트
   """
   result_documents = []
   
   for nid, doc in enumerate(documents, start=1):
       # 기존 메타데이터 복사
       new_metadata = doc.metadata.copy()
       
       # uuid와 nid 추가
       new_metadata['uuid'] = str(uuid.uuid4())
       new_metadata['nid'] = nid
       
       # 새로운 Document 생성
       new_doc = Document(
           page_content=doc.page_content,
           metadata=new_metadata
       )
       result_documents.append(new_doc)
   
   # 결과 확인을 위한 출력
   print(f"처리된 문서 수: {len(result_documents)}")
   if result_documents:
       print("\n첫 번째 문서의 식별자:")
       print(f"UUID: {result_documents[0].metadata['uuid']}")
       print(f"NID: {result_documents[0].metadata['nid']}")
       
       # 마지막 문서의 NID로 순차적 번호 부여 확인
       print(f"\n마지막 NID: {result_documents[-1].metadata['nid']}")
       
       # UUID 중복 검사
       uuids = [doc.metadata['uuid'] for doc in result_documents]
       unique_uuids = set(uuids)
       print(f"\nUUID 중복 여부: {len(uuids) != len(unique_uuids)}")
   
   return result_documents

# 실행
final_docs = add_unique_identifiers(enhanced_docs)

처리된 문서 수: 3629

첫 번째 문서의 식별자:
UUID: 5b6676aa-409a-42d5-9098-ad1c95b76cd5
NID: 1

마지막 NID: 3629

UUID 중복 여부: False


In [36]:
final_docs[-1].metadata['nid']

3629

----
## 전처리가 완료된 final_docs를 Pickle로 변환 

In [37]:
import pickle
import os

def save_documents_to_pickle(documents: List[Document], base_dir: str = "./data") -> str:
   """
   Documents를 pickle 파일로 저장
   
   Args:
       documents: 저장할 Document 리스트
       base_dir: 저장할 기본 디렉토리 경로
       
   Returns:
       저장된 pickle 파일의 경로
   """
   # 디렉토리가 없으면 생성
   os.makedirs(base_dir, exist_ok=True)
   
   # 현재 시간을 파일명에 포함
   filename = f"final_documents.pkl"
   filepath = os.path.join(base_dir, filename)
   
   # pickle 파일로 저장
   with open(filepath, 'wb') as f:
       pickle.dump(documents, f)
   
   print(f"처리된 문서 수: {len(documents)}")
   print(f"Pickle 파일 저장 경로: {filepath}")
   
   # 파일 크기 확인
   file_size = os.path.getsize(filepath) / (1024 * 1024)  # MB 단위로 변환
   print(f"파일 크기: {file_size:.2f} MB")
   
   return filepath

# 실행
pickle_path = save_documents_to_pickle(final_docs)

처리된 문서 수: 3629
Pickle 파일 저장 경로: ./data/final_documents.pkl
파일 크기: 165.94 MB
